# Pipeline Tutorial

This notebook serves as a tutorial for the `ugali` pipeline analysis. Its purpose is to document the steps taken to perform an end-to-end likelihood analysis on a raw data set.

In [ ]:
%matplotlib inline
import os

## Installing Test Data

First we create a working directory and download some test data:

In [ ]:
!mkdir -p work
os.chdir('./work')
!wget https://github.com/DarkEnergySurvey/ugali/releases/download/v1.7.0/ugali-test-data.tar.gz
!tar -xzf ugali-test-data.tar.gz && rm -f ugali-test-data.tar.gz

For easy access, we'll also create symlinks to some of the `ugali` components that we'll be using. 

In [ ]:
!rm -f ugali && ln -s ../ugali
!rm -f pipeline && ln -s ugali/pipeline
!rm -f config_test.yaml && ln -s ugali/config/config_test.yaml

Let's take a look at what we've got

In [ ]:
!ls -lh *

The components of our working directory are now:
* `healpix` - this directory contains the catalog data divided into files by healpix pixel (and labeled as such)
* `mask` - mask of the magnitude limit in "sparse" healpix representation corresponding to each catalog pixel
* `pipeline` - pipeline scripts
* `config_test.yaml` - pipeline configuration file

In [ ]:
import numpy as np
import healpy as hp
import fitsio
data = fitsio.read('healpix/catalog_hpx0687.fits')
print(data.dtype.names)
print(data[:3])

# Displaying the catalog as a healpix map
nside=4096
c = np.zeros(hp.nside2npix(nside))
pix,cts = np.unique(data['PIX4096'],return_counts=True)
c[pix] = cts

hp.cartview(c,lonra=[53.5,55],latra=[-55,-53.5])

In [ ]:
mask = fitsio.read('mask/maglim_g_hpx0687.fits')
print(mask.dtype.names)
print(mask[:3])

# Displaying the catalog as a healpix map
m = np.zeros(hp.nside2npix(nside))
m[mask['PIXEL']] = mask['MAGLIM']

hp.cartview(m,lonra=[53.5,55],latra=[-55,-53.5])

## Configuration 

The configuration file is the key to the ugali pipeline analysis. We are going to use [`config_test.yaml`](https://github.com/DarkEnergySurvey/ugali/blob/master/ugali/config/config_test.yaml). This file contains the path to the catalog and mask files that we will use, as well as other configuration parameters for the various pipeline steps.

In [ ]:
!head -n 25 ../ugali/config/config_test.yaml

## Likelihood Grid Scan

Our first step will be to perform a likelihood grid scan over our test data set. Before executing the script, let's figure out what the arguments are:

In [ ]:
!python pipeline/run_03.0_likelihood.py --help

The command that we will run is
```
python pipeline/run_03.0_likelihood.py config_test.yaml -r scan -q local
```
Breaking this down:
* `python pipeline/run_03.0_likelihood.py` - the script itself
* `config_test.yaml` - the first positional argument of all the pipeline scripts is the configuration file
* `-r scan` - the component of the pipeline we want to run; in this case the grid scan
* `-q local` - we are going to execute this component locally

Now to run the script...

In [ ]:
!python pipeline/run_03.0_likelihood.py config_test.yaml -r scan -q local

It will take a while for the script to run. The result will be a set of output files in the `scan` directory.

In [ ]:
!ls -lh scan

The next step is to merge the output likelihood files

In [ ]:
!python pipeline/run_03.0_likelihood.py config_test.yaml -r merge -q local

Then we can perform a search for over densities in the likelihood on the merged files.

In [ ]:
!ls pipeline

In [ ]:
!python pipeline/run_04.0_peak_finder.py config_test.yaml